# 고객 이탈 예측 - 데이터 전처리

이 노트북에서는 탐색적 데이터 분석(EDA) 결과를 바탕으로 머신러닝 모델 학습을 위한 데이터 전처리를 수행합니다.

## 주요 내용
1. **데이터 로드 및 검증**: EDA에서 분석한 데이터 불러오기
2. **데이터 정제**: 결측치, 이상치, 중복값 처리
3. **피처 엔지니어링**: 새로운 변수 생성 및 기존 변수 변환
4. **인코딩**: 범주형 변수 처리
5. **스케일링**: 수치형 변수 정규화
6. **데이터 분할**: 훈련/검증/테스트 세트 분리
7. **전처리된 데이터 저장**: 모델링을 위한 데이터 export

---

## 1. 라이브러리 및 설정

In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os

# 전처리 라이브러리
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 추가 유틸리티
import joblib
import pickle

# 설정
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# 한글 폰트 설정
plt.rcParams['font.family'] = ['AppleGothic'] if os.name == 'posix' else ['Malgun Gothic']
plt.rcParams['axes.unicode_minus'] = False

# 표시 옵션
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# 랜덤 시드
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("📚 전처리 라이브러리 로드 완료!")
print(f"🕐 전처리 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

📚 전처리 라이브러리 로드 완료!
🕐 전처리 시작 시간: 2025-08-03 18:53:31


## 2. 데이터 로드 및 검증

EDA에서 분석한 데이터를 불러오고 기본적인 검증을 수행합니다.

In [ ]:
# 데이터 파일 경로 확인
raw_data_path = '../data/raw/customer_data.csv'
processed_data_path = '../data/processed/'

# 폴더 생성
os.makedirs(processed_data_path, exist_ok=True)

# 데이터 로드 시도
if os.path.exists(raw_data_path):
    print(f"📁 데이터 로드: {raw_data_path}")
    df = pd.read_csv(raw_data_path)
    print("✅ 실제 데이터 로드 성공!")
else:
    print("⚠️ 실제 데이터 파일이 없습니다. 샘플 데이터를 생성합니다.")
    
    # 고객 이탈 예측을 위한 샘플 데이터 생성
    np.random.seed(RANDOM_STATE)
    n_samples = 5000
    
    # 고객 기본 정보
    customer_data = {
        'customer_id': [f'CUST_{i:05d}' for i in range(1, n_samples + 1)],
        'age': np.random.normal(45, 15, n_samples).astype(int),
        'gender': np.random.choice(['Male', 'Female'], n_samples, p=[0.52, 0.48]),
        'tenure_months': np.random.exponential(24, n_samples).astype(int),
        'monthly_charges': np.random.normal(65, 20, n_samples),
        'total_charges': None,  # 계산으로 생성
        'contract_type': np.random.choice(['Month-to-month', 'One year', 'Two year'], 
                                        n_samples, p=[0.55, 0.25, 0.20]),
        'payment_method': np.random.choice(['Electronic check', 'Credit card', 'Bank transfer', 'Mailed check'],
                                         n_samples, p=[0.35, 0.25, 0.25, 0.15]),
        'internet_service': np.random.choice(['DSL', 'Fiber optic', 'No'], 
                                           n_samples, p=[0.35, 0.45, 0.20]),
        'phone_service': np.random.choice(['Yes', 'No'], n_samples, p=[0.85, 0.15]),
        'multiple_lines': np.random.choice(['Yes', 'No', 'No phone service'], 
                                         n_samples, p=[0.40, 0.45, 0.15]),
        'online_security': np.random.choice(['Yes', 'No', 'No internet service'], 
                                          n_samples, p=[0.30, 0.50, 0.20]),
        'online_backup': np.random.choice(['Yes', 'No', 'No internet service'], 
                                        n_samples, p=[0.35, 0.45, 0.20]),
        'device_protection': np.random.choice(['Yes', 'No', 'No internet service'], 
                                            n_samples, p=[0.32, 0.48, 0.20]),
        'tech_support': np.random.choice(['Yes', 'No', 'No internet service'], 
                                       n_samples, p=[0.28, 0.52, 0.20]),
        'streaming_tv': np.random.choice(['Yes', 'No', 'No internet service'], 
                                       n_samples, p=[0.38, 0.42, 0.20]),
        'streaming_movies': np.random.choice(['Yes', 'No', 'No internet service'], 
                                           n_samples, p=[0.36, 0.44, 0.20]),
        'paperless_billing': np.random.choice(['Yes', 'No'], n_samples, p=[0.75, 0.25]),
        'senior_citizen': np.random.choice([0, 1], n_samples, p=[0.84, 0.16])
    }
    
    df = pd.DataFrame(customer_data)
    
    # total_charges 계산 (tenure_months * monthly_charges + 노이즈)
    df['total_charges'] = (df['tenure_months'] * df['monthly_charges'] + 
                          np.random.normal(0, 100, n_samples)).round(2)
    df['total_charges'] = np.maximum(df['total_charges'], df['monthly_charges'])  # 최소값 보정
    
    # 이탈 여부 생성 (다양한 요인을 고려한 확률적 생성)
    churn_prob = 0.15  # 기본 이탈 확률
    
    # 계약 유형에 따른 이탈 확률 조정
    contract_multiplier = df['contract_type'].map({
        'Month-to-month': 2.5,
        'One year': 1.0,
        'Two year': 0.3
    })
    
    # 기타 요인들
    age_factor = np.where(df['age'] > 65, 0.8, 1.0)  # 고령자 이탈 낮음
    tenure_factor = np.where(df['tenure_months'] < 6, 2.0, 
                           np.where(df['tenure_months'] > 36, 0.5, 1.0))
    charges_factor = np.where(df['monthly_charges'] > 80, 1.5, 0.8)
    
    # 최종 이탈 확률
    final_churn_prob = (churn_prob * contract_multiplier * age_factor * 
                       tenure_factor * charges_factor)
    final_churn_prob = np.clip(final_churn_prob, 0, 0.8)  # 최대 80% 제한
    
    df['churn'] = np.random.binomial(1, final_churn_prob)
    
    # 일부 결측치 의도적 생성
    missing_indices = np.random.choice(n_samples, int(n_samples * 0.02), replace=False)
    df.loc[missing_indices, 'total_charges'] = np.nan
    
    print("✅ 샘플 데이터 생성 완료!")

# 데이터 기본 정보 출력
print(f"\n📊 데이터 기본 정보:")
print(f"  • 전체 샘플 수: {len(df):,}")
print(f"  • 전체 피처 수: {df.shape[1]}")
print(f"  • 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# 타겟 변수 분포
if 'churn' in df.columns:
    churn_rate = df['churn'].mean()
    print(f"  • 이탈률: {churn_rate:.2%}")
    print(f"  • 이탈 고객 수: {df['churn'].sum():,}")
    print(f"  • 유지 고객 수: {(df['churn'] == 0).sum():,}")

print(f"\n📋 데이터 타입 정보:")
print(df.dtypes.value_counts())

print(f"\n🔍 데이터 미리보기:")
display(df.head())

## 3. 데이터 정제

결측치, 이상치, 중복값을 처리하고 데이터 품질을 개선합니다.

In [ ]:
# 데이터 정제 전 백업
df_original = df.copy()

print("🧹 데이터 정제 시작")
print("=" * 50)

# 1. 결측치 분석
print("📊 결측치 분석:")
missing_info = df.isnull().sum()
missing_percent = (missing_info / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing_Count': missing_info,
    'Missing_Percent': missing_percent
}).sort_values('Missing_Count', ascending=False)

# 결측치가 있는 컬럼만 표시
missing_cols = missing_df[missing_df['Missing_Count'] > 0]
if len(missing_cols) > 0:
    print("❌ 결측치가 있는 컬럼:")
    display(missing_cols)
else:
    print("✅ 결측치 없음")

# 2. 중복 데이터 확인
duplicates = df.duplicated().sum()
print(f"\n🔍 중복 데이터: {duplicates}개")

if duplicates > 0:
    print("❌ 중복 데이터 제거 중...")
    df = df.drop_duplicates()
    print(f"✅ {duplicates}개 중복 데이터 제거 완료")

# 3. 결측치 처리
if len(missing_cols) > 0:
    print(f"\n🔧 결측치 처리 시작...")
    
    for col in missing_cols.index:
        missing_count = missing_cols.loc[col, 'Missing_Count']
        missing_pct = missing_cols.loc[col, 'Missing_Percent']
        
        print(f"\n  📋 {col} 컬럼 처리 ({missing_count}개, {missing_pct}%)")
        
        if df[col].dtype in ['int64', 'float64']:  # 수치형 데이터
            if missing_pct < 5:  # 5% 미만이면 중앙값으로 대체
                median_value = df[col].median()
                df[col].fillna(median_value, inplace=True)
                print(f"    ✅ 중앙값({median_value:.2f})으로 대체")
            else:  # 5% 이상이면 KNN 대체
                # 수치형 컬럼들만 선택해서 KNN 적용
                numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
                if col in numeric_cols:
                    numeric_cols.remove(col)
                
                if len(numeric_cols) > 0:
                    imputer = KNNImputer(n_neighbors=5)
                    df[col] = imputer.fit_transform(df[[col] + numeric_cols[:3]])[:, 0]
                    print(f"    ✅ KNN 대체 완료")
                else:
                    median_value = df[col].median()
                    df[col].fillna(median_value, inplace=True)
                    print(f"    ✅ 중앙값({median_value:.2f})으로 대체")
        else:  # 범주형 데이터
            mode_value = df[col].mode().iloc[0] if len(df[col].mode()) > 0 else 'Unknown'
            df[col].fillna(mode_value, inplace=True)
            print(f"    ✅ 최빈값('{mode_value}')으로 대체")

# 4. 데이터 타입 최적화
print(f"\n🎯 데이터 타입 최적화:")

# 정수형 최적화
for col in df.select_dtypes(include=['int64']).columns:
    col_min, col_max = df[col].min(), df[col].max()
    if col_min >= 0:  # 양수인 경우
        if col_max < 255:
            df[col] = df[col].astype('uint8')
        elif col_max < 65535:
            df[col] = df[col].astype('uint16')
        elif col_max < 4294967295:
            df[col] = df[col].astype('uint32')
    else:  # 음수가 있는 경우
        if col_min > -128 and col_max < 127:
            df[col] = df[col].astype('int8')
        elif col_min > -32768 and col_max < 32767:
            df[col] = df[col].astype('int16')
        elif col_min > -2147483648 and col_max < 2147483647:
            df[col] = df[col].astype('int32')

# 실수형 최적화
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = pd.to_numeric(df[col], downcast='float')

# 5. 이상치 탐지 및 처리
print(f"\n🔎 이상치 탐지 (수치형 변수):")

numeric_columns = df.select_dtypes(include=[np.number]).columns
outlier_summary = {}

for col in numeric_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outlier_pct = (outliers / len(df) * 100).round(2)
    
    outlier_summary[col] = {
        'count': outliers,
        'percentage': outlier_pct,
        'lower_bound': lower_bound,
        'upper_bound': upper_bound
    }
    
    if outliers > 0:
        print(f"  📊 {col}: {outliers}개 ({outlier_pct}%)")
        
        # 이상치가 5% 미만이면 상한/하한선으로 클리핑
        if outlier_pct < 5:
            df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
            print(f"      ✅ 상한/하한선으로 클리핑 적용")

# 6. 정제 결과 요약
print(f"\n📋 데이터 정제 결과:")
print(f"  • 처리 전 샘플 수: {len(df_original):,}")
print(f"  • 처리 후 샘플 수: {len(df):,}")
print(f"  • 제거된 샘플 수: {len(df_original) - len(df):,}")

# 메모리 사용량 비교
memory_before = df_original.memory_usage(deep=True).sum() / 1024**2
memory_after = df.memory_usage(deep=True).sum() / 1024**2
memory_reduction = ((memory_before - memory_after) / memory_before * 100)

print(f"  • 메모리 사용량 (전): {memory_before:.2f} MB")
print(f"  • 메모리 사용량 (후): {memory_after:.2f} MB")
print(f"  • 메모리 절약: {memory_reduction:.1f}%")

# 최종 결측치 확인
final_missing = df.isnull().sum().sum()
print(f"  • 남은 결측치: {final_missing}개")

if final_missing == 0:
    print("✅ 모든 결측치 처리 완료!")
else:
    print("⚠️ 일부 결측치가 남아있습니다.")

print(f"\n🔍 정제된 데이터 미리보기:")
display(df.head())

## 4. 피처 엔지니어링

기존 변수를 활용하여 새로운 의미 있는 변수를 생성하고 기존 변수를 변환합니다.

In [ ]:
print("🔧 피처 엔지니어링 시작")
print("=" * 50)

# 1. 새로운 수치형 피처 생성
print("📊 수치형 피처 생성:")

# 고객 가치 관련 피처
if 'monthly_charges' in df.columns and 'tenure_months' in df.columns:
    # 월 평균 사용료 대비 총 사용료 비율
    df['charges_per_month'] = df['total_charges'] / (df['tenure_months'] + 1)  # +1로 0 나누기 방지
    print("  ✅ charges_per_month: 월평균 요금")
    
    # 고객 생애 가치 (Customer Lifetime Value 추정)
    df['estimated_clv'] = df['monthly_charges'] * df['tenure_months'] * 1.2  # 20% 마진 고려
    print("  ✅ estimated_clv: 추정 고객 생애 가치")

# 고객 세그먼트 관련 피처
if 'age' in df.columns:
    # 연령대 그룹
    df['age_group'] = pd.cut(df['age'], 
                            bins=[0, 25, 35, 45, 55, 65, 100], 
                            labels=['Under_25', '25-34', '35-44', '45-54', '55-64', 'Over_65'])
    print("  ✅ age_group: 연령대 그룹")

if 'tenure_months' in df.columns:
    # 고객 충성도 세그먼트
    df['loyalty_segment'] = pd.cut(df['tenure_months'],
                                  bins=[0, 6, 12, 24, 60, 1000],
                                  labels=['New', 'Growing', 'Established', 'Loyal', 'Champion'])
    print("  ✅ loyalty_segment: 고객 충성도 세그먼트")

# 2. 서비스 관련 피처 엔지니어링
print(f"\n🛠️ 서비스 관련 피처:")

# 부가 서비스 개수 카운트
service_cols = [col for col in df.columns if any(service in col.lower() 
                for service in ['security', 'backup', 'protection', 'support', 
                              'streaming', 'lines'])]

if service_cols:
    # 부가 서비스 중 'Yes'인 것의 개수
    def count_services(row):
        return sum(1 for col in service_cols if row[col] == 'Yes')
    
    df['total_services'] = df.apply(count_services, axis=1)
    print(f"  ✅ total_services: 이용 중인 부가 서비스 수 ({len(service_cols)}개 중)")

# 인터넷 서비스 품질 점수
if 'internet_service' in df.columns:
    internet_score_map = {'Fiber optic': 3, 'DSL': 2, 'No': 0}
    df['internet_score'] = df['internet_service'].map(internet_score_map)
    print("  ✅ internet_score: 인터넷 서비스 품질 점수")

# 3. 계약 관련 피처
print(f"\n📋 계약 관련 피처:")

if 'contract_type' in df.columns:
    # 계약 안정성 점수
    contract_stability = {'Month-to-month': 1, 'One year': 2, 'Two year': 3}
    df['contract_stability'] = df['contract_type'].map(contract_stability)
    print("  ✅ contract_stability: 계약 안정성 점수")

if 'payment_method' in df.columns:
    # 자동 결제 여부
    auto_payment = ['Credit card', 'Bank transfer']
    df['auto_payment'] = df['payment_method'].apply(lambda x: 1 if x in auto_payment else 0)
    print("  ✅ auto_payment: 자동 결제 여부")

# 4. 고객 리스크 점수 계산
print(f"\n⚠️ 리스크 점수 계산:")

risk_factors = []

# 계약 유형 리스크 (월 단위 계약이 가장 위험)
if 'contract_type' in df.columns:
    contract_risk = df['contract_type'].map({'Month-to-month': 3, 'One year': 2, 'Two year': 1})
    risk_factors.append(contract_risk)

# 신규 고객 리스크 (tenure가 짧을수록 위험)
if 'tenure_months' in df.columns:
    tenure_risk = np.where(df['tenure_months'] < 6, 3,
                          np.where(df['tenure_months'] < 12, 2, 1))
    risk_factors.append(pd.Series(tenure_risk, index=df.index))

# 고요금 고객 리스크 (월 요금이 높을수록 이탈 가능성 증가)
if 'monthly_charges' in df.columns:
    charges_percentile = df['monthly_charges'].rank(pct=True)
    charges_risk = np.where(charges_percentile > 0.8, 3,
                           np.where(charges_percentile > 0.6, 2, 1))
    risk_factors.append(pd.Series(charges_risk, index=df.index))

# 수동 결제 리스크
if 'auto_payment' in df.columns:
    payment_risk = np.where(df['auto_payment'] == 0, 2, 1)
    risk_factors.append(pd.Series(payment_risk, index=df.index))

# 종합 리스크 점수
if risk_factors:
    df['risk_score'] = sum(risk_factors)
    df['risk_level'] = pd.cut(df['risk_score'], 
                             bins=[0, 4, 7, 10, 20], 
                             labels=['Low', 'Medium', 'High', 'Critical'])
    print(f"  ✅ risk_score: 종합 리스크 점수 (4-{df['risk_score'].max()})")
    print(f"  ✅ risk_level: 리스크 레벨 (Low/Medium/High/Critical)")

# 5. 상호작용 피처 (중요한 변수들 간의 조합)
print(f"\n🔗 상호작용 피처:")

if 'monthly_charges' in df.columns and 'tenure_months' in df.columns:
    # 요금 대비 충성도
    df['charges_tenure_ratio'] = df['monthly_charges'] / (df['tenure_months'] + 1)
    print("  ✅ charges_tenure_ratio: 요금 대비 충성도")

if 'total_services' in df.columns and 'monthly_charges' in df.columns:
    # 서비스당 평균 요금
    df['charges_per_service'] = df['monthly_charges'] / (df['total_services'] + 1)
    print("  ✅ charges_per_service: 서비스당 평균 요금")

# 6. 로그 변환 (치우친 분포의 수치형 변수)
print(f"\n📈 로그 변환:")

log_transform_cols = []
for col in ['monthly_charges', 'total_charges', 'tenure_months']:
    if col in df.columns:
        # 치우침 정도 확인 (왜도)
        skewness = df[col].skew()
        if abs(skewness) > 1:  # 치우침이 심한 경우
            df[f'{col}_log'] = np.log1p(df[col])  # log1p는 log(1+x)로 0값 처리
            log_transform_cols.append(col)
            print(f"  ✅ {col}_log: {col}의 로그 변환 (원본 왜도: {skewness:.2f})")

# 7. 생성된 피처 요약
print(f"\n📊 피처 엔지니어링 결과:")
new_features = [col for col in df.columns if col not in df_original.columns]
print(f"  • 새로 생성된 피처 수: {len(new_features)}")
print(f"  • 전체 피처 수: {df.shape[1]} (기존 {df_original.shape[1]}개 + 신규 {len(new_features)}개)")

if new_features:
    print(f"  • 새로운 피처 목록:")
    for i, feature in enumerate(new_features, 1):
        print(f"    {i:2d}. {feature}")

# 데이터 타입별 피처 분포
print(f"\n📋 피처 타입 분포:")
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"  • {dtype}: {count}개")

print(f"\n✅ 피처 엔지니어링 완료!")
print(f"🔍 업데이트된 데이터 미리보기:")
display(df.head())

## 5. 인코딩

범주형 변수를 머신러닝 모델이 처리할 수 있는 수치형으로 변환합니다.

In [ ]:
print("🔤 범주형 변수 인코딩 시작")
print("=" * 50)

# 인코딩 전 데이터 백업
df_before_encoding = df.copy()

# 1. 범주형 변수 식별
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"📋 범주형 변수 ({len(categorical_columns)}개):")
for i, col in enumerate(categorical_columns, 1):
    unique_values = df[col].nunique()
    print(f"  {i:2d}. {col}: {unique_values}개 고유값")

# customer_id는 제외 (식별자이므로)
if 'customer_id' in categorical_columns:
    categorical_columns.remove('customer_id')
    print(f"  ℹ️ customer_id는 인코딩에서 제외")

# 2. 이진 인코딩 (Binary/Boolean 변수들)
print(f"\n🔢 이진 인코딩:")
binary_columns = []

for col in categorical_columns:
    unique_vals = df[col].unique()
    if len(unique_vals) == 2:
        binary_columns.append(col)
        # Yes/No, Male/Female 등을 0/1로 변환
        if 'Yes' in unique_vals and 'No' in unique_vals:
            df[col] = df[col].map({'Yes': 1, 'No': 0})
        elif 'Male' in unique_vals and 'Female' in unique_vals:
            df[col] = df[col].map({'Male': 1, 'Female': 0})
        else:
            # 기타 이진 변수는 LabelEncoder 사용
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
        
        print(f"  ✅ {col}: {unique_vals} → 0/1")

# 3. 서수 인코딩 (Ordinal Encoding) - 순서가 있는 범주형 변수
print(f"\n📊 서수 인코딩:")

ordinal_mappings = {}

# 계약 안정성 (이미 처리됨)
if 'contract_type' in df.columns and 'contract_stability' not in df.columns:
    contract_order = {'Month-to-month': 1, 'One year': 2, 'Two year': 3}
    df['contract_stability'] = df['contract_type'].map(contract_order)
    ordinal_mappings['contract_type'] = contract_order
    print(f"  ✅ contract_type → contract_stability: {contract_order}")

# 인터넷 서비스 품질 (이미 처리됨)
if 'internet_service' in df.columns and 'internet_score' not in df.columns:
    internet_order = {'No': 0, 'DSL': 1, 'Fiber optic': 2}
    df['internet_score'] = df['internet_service'].map(internet_order)
    ordinal_mappings['internet_service'] = internet_order
    print(f"  ✅ internet_service → internet_score: {internet_order}")

# 4. 원-핫 인코딩 (Nominal 변수들)
print(f"\n🎯 원-핫 인코딩:")

# 원-핫 인코딩할 변수들 (카디널리티가 낮은 명목형 변수)
onehot_columns = []
for col in categorical_columns:
    if col not in binary_columns:
        unique_count = df[col].nunique()
        if unique_count <= 10:  # 10개 이하의 카테고리만 원-핫 인코딩
            onehot_columns.append(col)

print(f"  📋 원-핫 인코딩 대상 ({len(onehot_columns)}개):")
for col in onehot_columns:
    print(f"    • {col}: {df[col].nunique()}개 카테고리")

# 원-핫 인코딩 수행
if onehot_columns:
    # pandas get_dummies 사용
    df_encoded = pd.get_dummies(df, columns=onehot_columns, prefix=onehot_columns, 
                               prefix_sep='_', drop_first=False)
    
    # 원래 컬럼들 제거
    df_encoded = df_encoded.drop(columns=onehot_columns)
    
    print(f"  ✅ 원-핫 인코딩 완료")
    print(f"    - 생성된 더미 변수: {len(df_encoded.columns) - len(df.columns)}개")
    
    df = df_encoded.copy()
else:
    print(f"  ℹ️ 원-핫 인코딩할 변수가 없습니다.")

# 5. 고카디널리티 변수 처리 (카테고리가 많은 변수)
print(f"\n🔍 고카디널리티 변수 처리:")

high_cardinality_cols = []
for col in categorical_columns:
    if col not in binary_columns and col not in onehot_columns:
        unique_count = df[col].nunique()
        if unique_count > 10:
            high_cardinality_cols.append(col)

if high_cardinality_cols:
    print(f"  📋 고카디널리티 변수 ({len(high_cardinality_cols)}개):")
    for col in high_cardinality_cols:
        unique_count = df[col].nunique()
        print(f"    • {col}: {unique_count}개 카테고리")
        
        # 타겟 인코딩 또는 빈도 인코딩 적용
        if 'churn' in df.columns:
            # 타겟 인코딩 (이탈률 기반)
            target_encoding = df.groupby(col)['churn'].mean()
            df[f'{col}_target_encoded'] = df[col].map(target_encoding)
            print(f"      ✅ 타겟 인코딩 적용 → {col}_target_encoded")
        
        # 빈도 인코딩
        frequency_encoding = df[col].value_counts()
        df[f'{col}_frequency'] = df[col].map(frequency_encoding)
        print(f"      ✅ 빈도 인코딩 적용 → {col}_frequency")
        
        # 원본 컬럼 제거
        df = df.drop(columns=[col])
else:
    print(f"  ℹ️ 고카디널리티 변수가 없습니다.")

# 6. 최종 범주형 변수 처리 확인
remaining_categorical = df.select_dtypes(include=['object', 'category']).columns.tolist()
if 'customer_id' in remaining_categorical:
    remaining_categorical.remove('customer_id')

if remaining_categorical:
    print(f"\n⚠️ 처리되지 않은 범주형 변수:")
    for col in remaining_categorical:
        print(f"  • {col}: {df[col].nunique()}개 고유값")
        # 남은 범주형 변수는 라벨 인코딩으로 처리
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        print(f"    ✅ 라벨 인코딩 적용")

# 7. 인코딩 결과 요약
print(f"\n📊 인코딩 결과 요약:")
print(f"  • 인코딩 전 컬럼 수: {df_before_encoding.shape[1]}")
print(f"  • 인코딩 후 컬럼 수: {df.shape[1]}")
print(f"  • 추가된 컬럼 수: {df.shape[1] - df_before_encoding.shape[1]}")

# 데이터 타입 분포
numeric_cols = len(df.select_dtypes(include=[np.number]).columns)
categorical_cols = len(df.select_dtypes(include=['object', 'category']).columns)

print(f"  • 수치형 변수: {numeric_cols}개")
print(f"  • 범주형 변수: {categorical_cols}개")

# 타겟 변수 확인
if 'churn' in df.columns:
    print(f"  • 타겟 변수 (churn): {df['churn'].dtype}")
    print(f"    - 클래스 분포: {df['churn'].value_counts().to_dict()}")

print(f"\n✅ 모든 인코딩 완료!")

# 인코딩된 데이터 미리보기
print(f"\n🔍 인코딩된 데이터 미리보기:")
display(df.head())

# 컬럼명 정리 (공백이나 특수문자 제거)
df.columns = df.columns.str.replace(' ', '_').str.replace('-', '_').str.lower()
print(f"\n🔧 컬럼명 정규화 완료 (소문자, 언더스코어 사용)")

## 6. 데이터 분할 및 저장

전처리가 완료된 데이터를 훈련/검증/테스트 세트로 분할하고 저장합니다.

In [ ]:
print("📊 데이터 분할 및 저장")
print("=" * 50)

# 1. 피처와 타겟 분리
if 'churn' in df.columns:
    X = df.drop(['churn'], axis=1)
    y = df['churn']
    
    # customer_id가 있으면 제거 (모델링에 불필요)
    if 'customer_id' in X.columns:
        customer_ids = X['customer_id'].copy()
        X = X.drop(['customer_id'], axis=1)
    else:
        customer_ids = None
    
    print(f"✅ 피처-타겟 분리 완료:")
    print(f"  • 피처 수 (X): {X.shape[1]}")
    print(f"  • 타겟 변수 (y): {y.name}")
    print(f"  • 총 샘플 수: {len(X)}")
else:
    print("❌ 타겟 변수 'churn'이 없습니다. 전체 데이터를 피처로 처리합니다.")
    X = df.copy()
    if 'customer_id' in X.columns:
        customer_ids = X['customer_id'].copy()
        X = X.drop(['customer_id'], axis=1)
    else:
        customer_ids = None
    y = None

# 2. 데이터 분할 (훈련:검증:테스트 = 70:15:15)
if y is not None:
    print(f"\n🔄 데이터 분할 시작:")
    print(f"  분할 비율: 훈련(70%) : 검증(15%) : 테스트(15%)")
    
    # 먼저 훈련+검증 vs 테스트로 분할 (85:15)
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.15, random_state=RANDOM_STATE, stratify=y
    )
    
    # 다시 훈련 vs 검증으로 분할 (70:15 = 82.35:17.65)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.176, random_state=RANDOM_STATE, stratify=y_temp
    )
    
    print(f"✅ 데이터 분할 완료:")
    print(f"  • 훈련 세트: {len(X_train):,}개 ({len(X_train)/len(X)*100:.1f}%)")
    print(f"  • 검증 세트: {len(X_val):,}개 ({len(X_val)/len(X)*100:.1f}%)")
    print(f"  • 테스트 세트: {len(X_test):,}개 ({len(X_test)/len(X)*100:.1f}%)")
    
    # 각 세트의 타겟 분포 확인
    print(f"\n📊 각 세트별 이탈률:")
    print(f"  • 전체: {y.mean():.3f}")
    print(f"  • 훈련: {y_train.mean():.3f}")
    print(f"  • 검증: {y_val.mean():.3f}")
    print(f"  • 테스트: {y_test.mean():.3f}")
    
    # customer_id도 함께 분할 (있는 경우)
    if customer_ids is not None:
        ids_temp = customer_ids.iloc[X_temp.index]
        ids_test = customer_ids.iloc[X_test.index]
        ids_train = ids_temp.iloc[X_train.index]
        ids_val = ids_temp.iloc[X_val.index]
else:
    print("⚠️ 타겟 변수가 없어 분할을 건너뜁니다.")
    X_train, X_val, X_test = X, None, None
    y_train, y_val, y_test = None, None, None

# 3. 스케일링 (훈련 세트 기준으로 fit, 모든 세트에 transform)
print(f"\n⚖️ 피처 스케일링:")

# 수치형 컬럼 식별
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
print(f"  • 스케일링 대상: {len(numeric_features)}개 수치형 피처")

if len(numeric_features) > 0:
    # StandardScaler 적용
    scaler = StandardScaler()
    
    # 훈련 세트로 scaler 학습
    X_train_scaled = X_train.copy()
    X_train_scaled[numeric_features] = scaler.fit_transform(X_train[numeric_features])
    
    if X_val is not None:
        X_val_scaled = X_val.copy()
        X_val_scaled[numeric_features] = scaler.transform(X_val[numeric_features])
    else:
        X_val_scaled = None
    
    if X_test is not None:
        X_test_scaled = X_test.copy()
        X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])
    else:
        X_test_scaled = None
    
    print(f"  ✅ StandardScaler 적용 완료")
    print(f"    - 평균: 0, 표준편차: 1로 정규화")
    
    # 스케일링 전후 비교 (첫 번째 수치형 피처)
    if len(numeric_features) > 0:
        sample_feature = numeric_features[0]
        print(f"    - 예시 ({sample_feature}):")
        print(f"      원본: 평균={X_train[sample_feature].mean():.2f}, 표준편차={X_train[sample_feature].std():.2f}")
        print(f"      변환: 평균={X_train_scaled[sample_feature].mean():.2f}, 표준편차={X_train_scaled[sample_feature].std():.2f}")
else:
    print("  ℹ️ 스케일링할 수치형 피처가 없습니다.")
    X_train_scaled = X_train.copy()
    X_val_scaled = X_val.copy() if X_val is not None else None
    X_test_scaled = X_test.copy() if X_test is not None else None
    scaler = None

# 4. 전처리된 데이터 저장
print(f"\n💾 전처리된 데이터 저장:")

# 저장 경로 설정
save_paths = {
    'train': os.path.join(processed_data_path, 'train_data.csv'),
    'val': os.path.join(processed_data_path, 'val_data.csv'),
    'test': os.path.join(processed_data_path, 'test_data.csv'),
    'train_scaled': os.path.join(processed_data_path, 'train_data_scaled.csv'),
    'val_scaled': os.path.join(processed_data_path, 'val_data_scaled.csv'),
    'test_scaled': os.path.join(processed_data_path, 'test_data_scaled.csv'),
    'scaler': os.path.join(processed_data_path, 'scaler.pkl'),
    'feature_names': os.path.join(processed_data_path, 'feature_names.pkl')
}

# 훈련 세트 저장
if y_train is not None:
    train_df = X_train.copy()
    train_df['churn'] = y_train
    train_df.to_csv(save_paths['train'], index=False)
    
    train_scaled_df = X_train_scaled.copy()
    train_scaled_df['churn'] = y_train
    train_scaled_df.to_csv(save_paths['train_scaled'], index=False)
    
    print(f"  ✅ 훈련 세트 저장: {save_paths['train']}")
    print(f"  ✅ 훈련 세트 (스케일링) 저장: {save_paths['train_scaled']}")

# 검증 세트 저장
if y_val is not None:
    val_df = X_val.copy()
    val_df['churn'] = y_val
    val_df.to_csv(save_paths['val'], index=False)
    
    val_scaled_df = X_val_scaled.copy()
    val_scaled_df['churn'] = y_val
    val_scaled_df.to_csv(save_paths['val_scaled'], index=False)
    
    print(f"  ✅ 검증 세트 저장: {save_paths['val']}")
    print(f"  ✅ 검증 세트 (스케일링) 저장: {save_paths['val_scaled']}")

# 테스트 세트 저장
if y_test is not None:
    test_df = X_test.copy()
    test_df['churn'] = y_test
    test_df.to_csv(save_paths['test'], index=False)
    
    test_scaled_df = X_test_scaled.copy()
    test_scaled_df['churn'] = y_test
    test_scaled_df.to_csv(save_paths['test_scaled'], index=False)
    
    print(f"  ✅ 테스트 세트 저장: {save_paths['test']}")
    print(f"  ✅ 테스트 세트 (스케일링) 저장: {save_paths['test_scaled']}")

# 스케일러 저장
if scaler is not None:
    with open(save_paths['scaler'], 'wb') as f:
        pickle.dump(scaler, f)
    print(f"  ✅ 스케일러 저장: {save_paths['scaler']}")

# 피처 이름 저장
feature_names = X_train.columns.tolist()
with open(save_paths['feature_names'], 'wb') as f:
    pickle.dump(feature_names, f)
print(f"  ✅ 피처 이름 저장: {save_paths['feature_names']}")

# 5. 전처리 요약 저장
summary_path = os.path.join(processed_data_path, 'preprocessing_summary.txt')
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write("고객 이탈 예측 - 데이터 전처리 요약\\n")
    f.write("=" * 50 + "\\n\\n")
    f.write(f"전처리 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n\\n")
    
    f.write(f"데이터 정보:\\n")
    f.write(f"  • 원본 샘플 수: {len(df_original):,}\\n")
    f.write(f"  • 최종 샘플 수: {len(df):,}\\n")
    f.write(f"  • 총 피처 수: {len(feature_names)}\\n")
    
    if y is not None:
        f.write(f"  • 이탈률: {y.mean():.3f}\\n")
        f.write(f"\\n데이터 분할:\\n")
        f.write(f"  • 훈련 세트: {len(X_train):,}개\\n")
        if X_val is not None:
            f.write(f"  • 검증 세트: {len(X_val):,}개\\n")
        if X_test is not None:
            f.write(f"  • 테스트 세트: {len(X_test):,}개\\n")
    
    f.write(f"\\n생성된 파일:\\n")
    for key, path in save_paths.items():
        if os.path.exists(path):
            f.write(f"  • {key}: {os.path.basename(path)}\\n")

print(f"  ✅ 전처리 요약 저장: {summary_path}")

# 6. 최종 결과 요약
print(f"\\n🎉 데이터 전처리 완료!")
print(f"=" * 60)
print(f"📊 최종 결과 요약:")
print(f"  • 처리된 총 샘플 수: {len(df):,}")
print(f"  • 최종 피처 수: {len(feature_names)}")
print(f"  • 저장된 파일 수: {len([p for p in save_paths.values() if os.path.exists(p)])}")
print(f"  • 저장 위치: {processed_data_path}")

if y is not None:
    print(f"\\n🎯 다음 단계:")
    print(f"  1. 03_Modeling.ipynb에서 모델 학습 진행")
    print(f"  2. 저장된 전처리 데이터 활용")
    print(f"  3. 스케일러를 사용한 새 데이터 전처리 가능")

print(f"\\n✅ 전처리 프로세스 성공적으로 완료!")
print(f"🕐 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)